# Scraping multiple pages

For this scraping excercise we'll use 3 libraries : `requests` and `Beautiful Soup` and `pandas`.
- `requests` : opens a website
- `BeautifulSoup` : parses the HTML
- `pandas` : for data analysis and transformation.
Let's import them.

- Go to the [Regulation No 31: laying down the Staff Regulations of Officials and the Conditions of Employment of Other Servants](https://eur-lex.europa.eu/legal-content/EN/TXT/?uri=CELEX%3A01962R0031-20230101).
- Look at the left-hand side side panel - it contains the older versions of this regulation. All of these contain this table. Let's collect all this data into one table.

## How:
- grab the links for the older regulations from the left-hand side panel
- write a `for` loop that loops over a range changing the URL for the regulations of previous years
- Reuse the script we wrote in the previous class to scrape all the pages. Put it into a function.
- Add a `year` column to the data so you know what year the allowances come from
- using [`pd.concat()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.concat.html) add all the tables into one
 

### There are multiple ways to find elements in a page

```python
soup.find('table') # returns the first occurence of the element `table` on the page
soup.find(string='text') # searches for text and returns the text !Does not return partial matches!
soup.find_all('div') # returns a list with all occurences of the element `div`
```

### You can also use CSS selectors
`soup.select()` always returns a list with all the occurences of the search, even if it's just a single element

```python
soup.select('a') #  all elements 'a'
soup.select('.classname') #  all elements with class name `.classname`
```

You will most ofthen combine them.

![html](img/css_selectors.png)

In [3]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs

## CSS selectors